<a href="https://colab.research.google.com/github/kvl38/Additional_education/blob/master/Kvl_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание для Junior DS по разработке PD модели

<b>Задание:</b> Необходиом разработать модель предсказания дефолта. Даны финансовые данные 32 395 компаний (выручка, активы, пассивы и т.д). Результат будет оцениваться на скрытой выборке из 200 компаний по метрике качества accuracy_score

<b>Цель задания</b> научиться / показать знания в разработке моделей оценки риска - probability of default. Задание приближено к реальному, данные синтетические. Лидерборд для оценки собственного решения и геймификации процесса обучения. Первое место в лидерборде не явлется целью. Цель - правильно разарботанная модель, удовлетворяющая требованиям банковской системы. В случае возникновения вопросов, сотрудники блока Риски готовы проконсультировать и дать обратную связь на модель (телеграм чат).

<b>Метрика качества</b> accuracy_score вычисляется, как ( True positive + True negative ) / Total. Если сабмит-файл, не правильного формата или неверное количество строк, метрика будет = 0.

### Импортируем библиотеки

In [158]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor

### Подгружаем данные

In [ ]:
!wget https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-train.csv
!wget https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-test.csv
!wget https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-desc.csv

In [275]:
train_df = pd.read_csv('PD-data-train.csv', sep=';')
test_df = pd.read_csv('PD-data-test.csv', sep=';')
desc_df = pd.read_csv('PD-data-desc.csv', sep=';')

Изучаем описание данных

In [ ]:
desc_df[:]

In [ ]:
train_df[:]

Здесь я искал зависимые друг от друга параметры.

(5) Прибыль от реализации = (0) Выручка от реализации - (1) Общие расходы за последний год

(5) ar_sale_profit = (0) ar_revenue - (1) ar_total_expenses

-------------------------------------------------------------------------------------
(1) Общие расходы за последний год = (2)Себестоимость продаж + (3) Коммерческие расходы + (4) Управленческие расходы

(1) ar_total_expenses = (2) ar_sale_cost + (3) ar_selling_expenses + (4) ar_management_expenses

-------------------------------------------------------------------------------------
(6) Баланс доходов и расходов = (7) Валовая прибыль - (5) Прибыль от реализации

(6) ar_balance_of_rvns_and_expns = (7) ar_profit_before_tax - (5) ar_sale_profit

-------------------------------------------------------------------------------------
(10) Чистая прибыль = (7) Валовая прибыль - (8) Текущий налог на прибыль + (9) Прочие доходы и расходы

(10) ar_net_profit = (7) ar_profit_before_tax - (8) ar_taxes + (9) ar_other_profit_and_losses

-------------------------------------------------------------------------------------
Независимые данные:

(11) Основные внеоборотные средства за последний год
(11) ab_immobilized_assets

(16) Денежные средства и денежные эквиваленты
(16) ab_cash_and_securities

(18) Капитал
(18) ab_own_capital

-------------------------------------------------------------------------------------
(24) Срок ведения бизнеса
(24) bus_age

(25) Срок с момента присваивания ОГРН
(25) ogrn_age

(24)bus_age = (24)ogrn_age

---------------------
(12) Оборотные активы = (13) Запасы  + (14) Дебиторская задолженность + (15) Прочие оборотные активы

(12) ab_mobile_current_assets = (13) ab_inventory + (14) ab_accounts_receivable + (15) ab_other_current_assets

-------------------------------------------------------------------------------------
(17) Нераспределенная прибыль = (10) Чистая прибыль

(17) ab_losses = (10) ar_net_profit 

-------------------------------------------------------------------------------------
(19) Общие долговые обязательства = (20) Долгосрочный обязательства + (21) Краткосрочные заемные средства + (22) Кредиторская задодженность за последний год + (23) Прочие обязательства

(19) ab_borrowed_capital = (20) ab_long_term_liabilities + (21) ab_short_term_borrowing + (22) ab_accounts_payable + (23) ab_other_borrowings

-------------------------------------------------------------------------------------

(26) adr_actual_age = (27)head_actual_age = (28)cap_actual_age

(26) Срок с момента регистрации юридического адреса
(26) adr_actual_age


(27) Срок с момента назначения руководителя
(27) head_actual_age


(28) Срок с момента установки капитала
(28) cap_actual_age

-------------------------------------------------------------------------------------
(29) Количество сотрудников
(29) ul_staff_range

-------------------------------------------------------------------------------------
(30) Величина капитала
(30) ul_capital_sum

-------------------------------------------------------------------------------------
(31) Количество акционеров
(31) ul_founders_cnt

-------------------------------------------------------------------------------------
(32) Количество филиалов
(32) ul_branch_cnt

-------------------------------------------------------------------------------------
(33) Компания стратегического значения
(33) ul_strategic_flg


-------------------------------------------------------------------------------------
(34) Системообразующая компания
(34) ul_systematizing_flg

train_df['ul_systematizing_flg'].value_counts()  => 

0    32395

Ответ: убираем

-------------------------------------------------------------------------------------
train_df.loc[(train_df['default_12m'] == 1)]

Всего: 2091 компания

На основе полученных данных я выбрал наиболее важные параметры

In [276]:
features = ['ar_sale_profit', 'ar_net_profit', 'ab_immobilized_assets', 'ab_mobile_current_assets', 'ab_cash_and_securities',
            'ab_own_capital', 'ab_borrowed_capital', 'bus_age', 'adr_actual_age', 'ul_capital_sum', 'ul_staff_range']


Бэйзлайн модель

In [ ]:
features_df = train_df.loc[:,features]
features_df['default_12m'] = train_df['default_12m']
features_df.head()

Строим карту корреляции

In [ ]:
features_df.corr()

На основе карты корреляции я решил убрать некоторые параметры

In [279]:
features_df.drop(['bus_age', 'adr_actual_age', 'ab_cash_and_securities'], axis=1, inplace=True)

Разбиваем параметр 'ul_staff_range' по трём столбцам:

ul_staff_range_(100-500] |	ul_staff_range_> 500 |	ul_staff_range_[1-100]

In [280]:
features_df = pd.get_dummies(features_df)

Убираем строки, в которых большая часть значений пропущена. (В строках находятся значения NaN)

In [281]:
my_df = features_df.dropna(thresh=6)

In [ ]:
my_df.head()

Разбиваем тренировочную выборку на обучающую и тестовую: (80/20)

In [282]:
X_train, X_test, y_train, y_test = train_test_split(my_df.drop('default_12m',axis=1), my_df['default_12m'], random_state=42, test_size=0.2)

При выборе лучшей модели были проверены: 

1) Логистическая регрессия

2) Градиентный бустинг

3) Случайный лес

Лучшее качество на обучающей выборке показал градиентный бустинг

Для подбора оптимальныъ параметров использовал GridSearchCV

Стандартизируем данные и находим точность модели на обучающей выборке

In [283]:
X_train = StandardScaler().fit_transform(X_train)

number_of_trees = 35 # оптимальное значение!
max_depth = 5 # оптимальное значение!
model = GradientBoostingClassifier(n_estimators=number_of_trees, max_depth=max_depth)
model_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc').mean()
print("ROC-AUC модели = {:.2f}%".format(model_scores * 100))

ROC-AUC модели = 70.06%


Построение модели на всех данных

In [284]:
model.fit(my_df.drop('default_12m',axis=1), my_df['default_12m'])

GradientBoostingClassifier(max_depth=5, n_estimators=35)

### Формирование файла с ответом

In [ ]:
answ_df = test_df[['record_id']]
answ_df['id'] = answ_df['record_id']
answ_df.drop('record_id',axis=1, inplace=True)
answ_df[:]

In [269]:
cutoff = y_train.mean()
cutoff

0.04601449275362319

Преобразуем тестовую выборку под полученную модель

In [ ]:
test_df = test_df.loc[:,features]
test_df.drop(['bus_age', 'adr_actual_age', 'ab_cash_and_securities'], axis=1, inplace=True)
test_df = pd.get_dummies(test_df)
test_df[:]

Заполняем все пропущенные значения в тестовой выборке нулями

In [271]:
test_df = test_df.replace(np.nan, 0)

В файл с ответом помещаем предсказанный нашей моделью результат

In [272]:
answ_df['predict'] = (model.predict_proba(test_df)[:,1]>cutoff).astype(int)

Сохраняем файл с ответом

In [273]:
answ_df.to_csv('kovalevPA.csv',index=False, sep=';')